# Spectral wave attenuation for the winter deployment
In this notebook, we derive frequency dependent wave attenuation using data from the winter deployment. During this deployment both buoys were in sea ice the whole time, and the wave activity was high enough to measure (significant wave height \> 0.1 m) during the whole deployment. We derive the attenuation assuming an exponential decay. For each pair of data points, the attenuation coefficient $\alpha$ is computed with
$$
\alpha(f) = \frac{\ln(E_\text{SWIFT20}(f) - E_\text{SWIFT21}(f)}{d}
$$
where $f$ is the frequency, $E$ the power spectral density and $d$ is the projected distance between the buoys in wave direction. For computing $d$, the mean wave direction of the two buoys is used.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

import waves
import attenuation as at

rc('font', size = 14)

In [ ]:
swift20_file = "../processed_data/swift20_winter.feather"
swift21_file = "../processed_data/swift21_winter.feather"

frequency_range = [0.05, 0.13] #Hz

### Get wave properties from both buoys:

In [ ]:
swift20 = pd.read_feather(swift20_file)
swift21 = pd.read_feather(swift21_file)

In [ ]:
tmin = max(swift20.timestamp[0], swift21.timestamp[0])
tmax = min(swift20.timestamp[len(swift20)-1], swift21.timestamp[len(swift21)-1])

swift20_pruned = swift20[(swift20.timestamp >= tmin) & (swift20.timestamp <= tmax)].reset_index()
swift21_pruned = swift21[(swift21.timestamp >= tmin) & (swift21.timestamp <= tmax)].reset_index()

### Compute pointwise attenuation
To compute the projected distance between the two buoys, we use the mean wave direction. To check that this is reasonable, we first plot the wave direction measured by the two buoys.

In [ ]:
plt.plot(swift20_pruned['timestamp'], swift20_pruned['swell.meandirT'])
plt.plot(swift21_pruned['timestamp'], swift21_pruned['swell.meandirT'])
plt.plot(swift20_pruned['timestamp'], (swift20_pruned['swell.meandirT']+swift21_pruned['swell.meandirT'])/2)
plt.ylabel('direction (deg)')
plt.grid()
plt.title('Swell wave direction')
plt.legend(['swift20', 'swift21', 'mean'])
plt.show()

We are now ready to compute the frequency dependent attenuation

In [ ]:
distance = at.distance_in_wave_direction(swift20_pruned, swift21_pruned)
swift20_spectra = waves.get_wavespectra(swift20_pruned)
swift21_spectra = waves.get_wavespectra(swift21_pruned)

ds = at.get_pointwise_attenuation(swift21_spectra, swift20_spectra, frequency_range, distance)
ds.alpha.plot()

### Fit to power law
We now fit the pointwise attenuation to a power law
$$
\alpha(f) = a \cdot f^b
$$
using non-linear least squares. We will do this both with and without constraints on $b$.

In [ ]:
# Initial parameter guess
a0 = 1
b0 = 3

popt, pcov = at.fit_alpha(ds.alpha, a0,b0)
popt_constr, pcov_constr = at.fit_alpha(ds.alpha, a0, b0, b_constraint = [2,4])

print("Power law fit: a = %.2f, b = %.2f" % (popt[0], popt[1]))
print("with constraints: a = %.2f, b = %.2f" % (popt_constr[0], popt_constr[1]))

## Box-and-whisker plot

In [ ]:
at.alpha_boxplot(ds.alpha)
at.plot_fit(popt, label=r'$af^b$')
at.plot_fit(popt_constr,
             label=r'$af^b$, $2\leq b\leq 4]$',
             linestyle='dashed')

plt.legend(loc = 'lower right')
plt.grid()